In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.svm import SVC

In [2]:
df_train=pd.read_csv('cc_train.csv')
df_test=pd.read_csv('cc_test.csv')

In [4]:
df_train.head()

,LIMIT_BAL,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,...,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,GENDER,QUALIFICATION,MARRIED,defaulter,Id
0,290000.0,50,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,199.0,...,1440.0,1017.0,3798.0,4898,5034.0,FEMALE,2,1.0,0,1
1,20000.0,49,0.0,0.0,0.0,0.0,NaN,0.0,19773.0,18459.0,...,1270.0,633.0,655.0,759,547.0,FEMALE,2,3.0,1,2
2,230000.0,28,0.0,0.0,0.0,0.0,0.0,0.0,20356.0,21504.0,...,1700.0,2007.0,1000.0,1000,1000.0,FEMALE,1,2.0,0,3
3,210000.0,31,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,8600.0,0.0,...,4231.0,4221.0,4221.0,4483,4300.0,FEMALE,1,1.0,0,4
4,300000.0,24,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-158.0,-158.0,...,0.0,0.0,300.0,100,100.0,MALE,2,2.0,0,5


In [4]:
df_train['QUALIFICATION'].value_counts()

2    9371
1    7072
3    3256
5     180
4      81
6      34
0       6
Name: QUALIFICATION, dtype: int64

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   LIMIT_BAL      18700 non-null  float64
 1   AGE            20000 non-null  int64  
 2   PAY_0          19344 non-null  float64
 3   PAY_2          18141 non-null  float64
 4   PAY_3          18751 non-null  float64
 5   PAY_4          19316 non-null  float64
 6   PAY_5          19323 non-null  float64
 7   PAY_6          18655 non-null  float64
 8   BILL_AMT1      18130 non-null  float64
 9   BILL_AMT2      18681 non-null  float64
 10  BILL_AMT3      19323 non-null  float64
 11  BILL_AMT4      20000 non-null  int64  
 12  BILL_AMT5      19318 non-null  float64
 13  BILL_AMT6      20000 non-null  int64  
 14  PAY_AMT1       18690 non-null  float64
 15  PAY_AMT2       18763 non-null  float64
 16  PAY_AMT3       18691 non-null  float64
 17  PAY_AMT4       19318 non-null  float64
 18  PAY_AM

In [6]:
df_train.columns

Index(['LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5',
       'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4',
       'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3',
       'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'GENDER', 'QUALIFICATION',
       'MARRIED', 'defaulter', 'Id'],
      dtype='object')

# Outlier Detection

In [7]:
bins=np.linspace(df_train['LIMIT_BAL'].min(),df_train['LIMIT_BAL'].max(),6)

In [8]:
bins

array([  10000.,  208000.,  406000.,  604000.,  802000., 1000000.])

In [9]:
z_Score = (df_train['defaulter'] - df_train['defaulter'].mean())/df_train['defaulter'].std()
    
df_train = df_train[((z_Score >-3) & (z_Score <=3))]

In [10]:
df_train

,LIMIT_BAL,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,...,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,GENDER,QUALIFICATION,MARRIED,defaulter,Id
0,290000.0,50,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,199.0,...,1440.0,1017.0,3798.0,4898,5034.0,FEMALE,2,1.0,0,1
1,20000.0,49,0.0,0.0,0.0,0.0,NaN,0.0,19773.0,18459.0,...,1270.0,633.0,655.0,759,547.0,FEMALE,2,3.0,1,2
2,230000.0,28,0.0,0.0,0.0,0.0,0.0,0.0,20356.0,21504.0,...,1700.0,2007.0,1000.0,1000,1000.0,FEMALE,1,2.0,0,3
3,210000.0,31,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,8600.0,0.0,...,4231.0,4221.0,4221.0,4483,4300.0,FEMALE,1,1.0,0,4
4,300000.0,24,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-158.0,-158.0,...,0.0,0.0,300.0,100,100.0,MALE,2,2.0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,310000.0,45,0.0,0.0,0.0,NaN,0.0,0.0,91532.0,93351.0,...,4370.0,4000.0,5000.0,3899,3761.0,FEMALE,1,1.0,0,19996
19996,180000.0,31,0.0,NaN,0.0,0.0,0.0,0.0,26000.0,26310.0,...,1500.0,1056.0,950.0,1000,NaN,FEMALE,2,1.0,0,19997
19997,210000.0,22,0.0,0.0,0.0,0.0,0.0,0.0,142848.0,94018.0,...,4415.0,6000.0,3635.0,4000,5000.0,FEMALE,2,1.0,0,19998
19998,200000.0,41,NaN,0.0,0.0,0.0,0.0,0.0,113570.0,113974.0,...,5571.0,5221.0,4001.0,4001,4001.0,FEMALE,3,1.0,0,19999


In [11]:
df_train.isna().sum()[df_train.isna().sum()>0]

LIMIT_BAL    1300
PAY_0         656
PAY_2        1859
PAY_3        1249
PAY_4         684
PAY_5         677
PAY_6        1345
BILL_AMT1    1870
BILL_AMT2    1319
BILL_AMT3     677
BILL_AMT5     682
PAY_AMT1     1310
PAY_AMT2     1237
PAY_AMT3     1309
PAY_AMT4      682
PAY_AMT6      688
MARRIED      1326
dtype: int64

In [12]:
df_test.isna().sum()[df_test.isna().sum()>0]

LIMIT_BAL    320
PAY_0        164
PAY_2        465
PAY_3        328
PAY_4        169
PAY_5        171
PAY_6        299
BILL_AMT1    465
BILL_AMT2    318
BILL_AMT3    174
BILL_AMT5    161
PAY_AMT1     349
PAY_AMT2     302
PAY_AMT3     368
PAY_AMT4     161
PAY_AMT6     192
MARRIED      321
dtype: int64

In [13]:
columns_cat = list(df_train.select_dtypes(include='object').columns)
columns_num = list(df_train.select_dtypes(exclude='object').columns)
columns_num.remove('Id')
columns_num.remove('defaulter')


# SimpleImputer

In [14]:
from sklearn.impute import SimpleImputer

In [15]:
imputer_num = SimpleImputer(strategy='median')
imputer_cat = SimpleImputer(strategy='most_frequent')

imputer_num.fit(df_train[columns_num])
imputer_cat.fit(df_train[columns_cat])

df_train[columns_num] = imputer_num.transform(df_train[columns_num])
df_train[columns_cat] = imputer_cat.transform(df_train[columns_cat])

df_test[columns_num] = imputer_num.transform(df_test[columns_num])
df_test[columns_cat] = imputer_cat.transform(df_test[columns_cat])

In [16]:
df_train.isna().sum()[df_train.isna().sum()>0], df_test.isna().sum()[df_test.isna().sum()>0]

(Series([], dtype: int64), Series([], dtype: int64))

# Scalling - MinMaxScaler

In [17]:
from sklearn.preprocessing import MinMaxScaler

In [18]:
scaler = MinMaxScaler()
scaler.fit(df_train[columns_num])
df_train[columns_num] = scaler.transform(df_train[columns_num])
df_test[columns_num] = scaler.transform(df_test[columns_num])

# OneHotEnoding

In [19]:
from sklearn.preprocessing import OneHotEncoder

In [20]:
ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(df_train[columns_cat])
df_train[ohe.get_feature_names()] = ohe.transform(df_train[columns_cat]).toarray()
df_test[ohe.get_feature_names()] = ohe.transform(df_test[columns_cat]).toarray()

C:\Users\ajay sutar\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\ajay sutar\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [21]:
ohe.transform(df_train[columns_cat]).toarray()

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [22]:
df_train.shape, df_test.shape

((20000, 27), (5000, 26))

# Feature Selection

In [23]:
col_list = columns_num + list(ohe.get_feature_names()) + ['defaulter']

C:\Users\ajay sutar\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [24]:
corr_values = df_train[col_list].corr()['defaulter']

In [25]:
selected_cols = list((corr_values[(corr_values>0.1) | (corr_values<-0.1)]).index)
selected_cols.remove('defaulter')

In [26]:
ohe.get_feature_names()

C:\Users\ajay sutar\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


array(['x0_FEMALE', 'x0_MALE'], dtype=object)

In [27]:
X = df_train[selected_cols]
y = df_train['defaulter']

# Model Building

In [28]:
from sklearn.ensemble import RandomForestClassifier

In [29]:
#model = RandomForestClassifier(n_estimators=10,random_state=95)
model=RandomForestClassifier(max_depth=11, min_samples_leaf=15, n_estimators=48,
                       random_state=95)
model.fit(X,y)


RandomForestClassifier(max_depth=11, min_samples_leaf=15, n_estimators=48,
                       random_state=95)

In [30]:
model.score(X,y)

0.8212

In [31]:
yp = model.predict(df_test[selected_cols])

df_test['defaulter'] = yp
df_test[['Id','defaulter']].to_csv('subRF6.csv',index=False)


In [32]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score

In [33]:
params = {'n_estimators':[10,20,30,40,50], 'max_depth':[2,3,4],'min_samples_leaf':[2,3,4,5]}
gridcv = GridSearchCV(RandomForestClassifier(random_state=95),
                      param_grid=params,cv=5,verbose=3, scoring='accuracy')
gridcv.fit(X,y)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV 1/5] END max_depth=2, min_samples_leaf=2, n_estimators=10;, score=0.803 total time=   0.0s
[CV 2/5] END max_depth=2, min_samples_leaf=2, n_estimators=10;, score=0.808 total time=   0.0s
[CV 3/5] END max_depth=2, min_samples_leaf=2, n_estimators=10;, score=0.803 total time=   0.0s
[CV 4/5] END max_depth=2, min_samples_leaf=2, n_estimators=10;, score=0.801 total time=   0.0s
[CV 5/5] END max_depth=2, min_samples_leaf=2, n_estimators=10;, score=0.808 total time=   0.0s
[CV 1/5] END max_depth=2, min_samples_leaf=2, n_estimators=20;, score=0.802 total time=   0.0s
[CV 2/5] END max_depth=2, min_samples_leaf=2, n_estimators=20;, score=0.807 total time=   0.0s
[CV 3/5] END max_depth=2, min_samples_leaf=2, n_estimators=20;, score=0.803 total time=   0.0s
[CV 4/5] END max_depth=2, min_samples_leaf=2, n_estimators=20;, score=0.799 total time=   0.0s
[CV 5/5] END max_depth=2, min_samples_leaf=2, n_estimators=20;, score=0.808 total t

[CV 4/5] END max_depth=2, min_samples_leaf=5, n_estimators=30;, score=0.801 total time=   0.0s
[CV 5/5] END max_depth=2, min_samples_leaf=5, n_estimators=30;, score=0.806 total time=   0.0s
[CV 1/5] END max_depth=2, min_samples_leaf=5, n_estimators=40;, score=0.807 total time=   0.0s
[CV 2/5] END max_depth=2, min_samples_leaf=5, n_estimators=40;, score=0.806 total time=   0.0s
[CV 3/5] END max_depth=2, min_samples_leaf=5, n_estimators=40;, score=0.807 total time=   0.0s
[CV 4/5] END max_depth=2, min_samples_leaf=5, n_estimators=40;, score=0.804 total time=   0.0s
[CV 5/5] END max_depth=2, min_samples_leaf=5, n_estimators=40;, score=0.808 total time=   0.0s
[CV 1/5] END max_depth=2, min_samples_leaf=5, n_estimators=50;, score=0.806 total time=   0.1s
[CV 2/5] END max_depth=2, min_samples_leaf=5, n_estimators=50;, score=0.808 total time=   0.1s
[CV 3/5] END max_depth=2, min_samples_leaf=5, n_estimators=50;, score=0.806 total time=   0.1s
[CV 4/5] END max_depth=2, min_samples_leaf=5, n_es

[CV 1/5] END max_depth=3, min_samples_leaf=5, n_estimators=10;, score=0.813 total time=   0.0s
[CV 2/5] END max_depth=3, min_samples_leaf=5, n_estimators=10;, score=0.812 total time=   0.0s
[CV 3/5] END max_depth=3, min_samples_leaf=5, n_estimators=10;, score=0.809 total time=   0.0s
[CV 4/5] END max_depth=3, min_samples_leaf=5, n_estimators=10;, score=0.805 total time=   0.0s
[CV 5/5] END max_depth=3, min_samples_leaf=5, n_estimators=10;, score=0.811 total time=   0.0s
[CV 1/5] END max_depth=3, min_samples_leaf=5, n_estimators=20;, score=0.811 total time=   0.0s
[CV 2/5] END max_depth=3, min_samples_leaf=5, n_estimators=20;, score=0.811 total time=   0.0s
[CV 3/5] END max_depth=3, min_samples_leaf=5, n_estimators=20;, score=0.810 total time=   0.0s
[CV 4/5] END max_depth=3, min_samples_leaf=5, n_estimators=20;, score=0.805 total time=   0.0s
[CV 5/5] END max_depth=3, min_samples_leaf=5, n_estimators=20;, score=0.810 total time=   0.0s
[CV 1/5] END max_depth=3, min_samples_leaf=5, n_es

[CV 3/5] END max_depth=4, min_samples_leaf=4, n_estimators=30;, score=0.819 total time=   0.0s
[CV 4/5] END max_depth=4, min_samples_leaf=4, n_estimators=30;, score=0.810 total time=   0.0s
[CV 5/5] END max_depth=4, min_samples_leaf=4, n_estimators=30;, score=0.818 total time=   0.0s
[CV 1/5] END max_depth=4, min_samples_leaf=4, n_estimators=40;, score=0.818 total time=   0.1s
[CV 2/5] END max_depth=4, min_samples_leaf=4, n_estimators=40;, score=0.821 total time=   0.1s
[CV 3/5] END max_depth=4, min_samples_leaf=4, n_estimators=40;, score=0.820 total time=   0.0s
[CV 4/5] END max_depth=4, min_samples_leaf=4, n_estimators=40;, score=0.809 total time=   0.1s
[CV 5/5] END max_depth=4, min_samples_leaf=4, n_estimators=40;, score=0.820 total time=   0.1s
[CV 1/5] END max_depth=4, min_samples_leaf=4, n_estimators=50;, score=0.817 total time=   0.1s
[CV 2/5] END max_depth=4, min_samples_leaf=4, n_estimators=50;, score=0.820 total time=   0.1s
[CV 3/5] END max_depth=4, min_samples_leaf=4, n_es

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=95),
             param_grid={'max_depth': [2, 3, 4],
                         'min_samples_leaf': [2, 3, 4, 5],
                         'n_estimators': [10, 20, 30, 40, 50]},
             scoring='accuracy', verbose=3)

In [34]:
gridcv.best_params_

{'max_depth': 4, 'min_samples_leaf': 2, 'n_estimators': 40}

In [35]:
gridcv.best_estimator_

RandomForestClassifier(max_depth=4, min_samples_leaf=2, n_estimators=40,
                       random_state=95)

In [36]:
gridcv.best_score_

0.8179500000000001

In [37]:
final_model = RandomForestClassifier(max_depth=5, min_samples_leaf=5, min_samples_split=6,n_estimators=50)

In [38]:
final_model.fit(X,y)

RandomForestClassifier(max_depth=5, min_samples_leaf=5, min_samples_split=6,
                       n_estimators=50)

In [39]:
yp = final_model.predict(df_test[selected_cols])

df_test['defaulter'] = yp
#df_test[['Id','defaulter']].to_csv('sub4.csv',index=False)

In [40]:
from sklearn.tree import DecisionTreeClassifier

In [41]:
model2 = DecisionTreeClassifier()
model2.fit(X,y)

DecisionTreeClassifier()

In [42]:
model2.score(X,y)

0.87915

In [43]:
yp = model2.predict(df_test[selected_cols])

df_test['defaulter'] = yp
df_test[['Id','defaulter']].to_csv('subDT5.csv',index=False)